In [1]:
import findspark
findspark.init()

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType


spark = SparkSession.builder.appName("inference").getOrCreate()

:: loading settings :: url = jar:file:/Users/danilonajkov/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/danilonajkov/.ivy2/cache
The jars for the packages stored in: /Users/danilonajkov/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6e19a816-30e1-42d7-859c-e64f85667a7c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 226ms :: 

In [5]:
loaded_model = PipelineModel.load("best_model")

schema = StructType([
    StructField("Id", DoubleType()),
    StructField("Diabetes_binary", DoubleType()),
    StructField("HighBP", DoubleType()),
    StructField("HighChol", DoubleType()),
    StructField("CholCheck", DoubleType()),
    StructField("BMI", DoubleType()),
    StructField("Smoker", DoubleType()),
    StructField("Stroke", DoubleType()),
    StructField("HeartDiseaseorAttack", DoubleType()),
    StructField("PhysActivity", DoubleType()),
    StructField("Fruits", DoubleType()),
    StructField("Veggies", DoubleType()),
    StructField("HvyAlcoholConsump", DoubleType()),
    StructField("AnyHealthcare", DoubleType()),
    StructField("NoDocbcCost", DoubleType()),
    StructField("GenHlth", DoubleType()),
    StructField("MentHlth", DoubleType()),
    StructField("PhysHlth", DoubleType()),
    StructField("DiffWalk", DoubleType()),
    StructField("Sex", DoubleType()),
    StructField("Age", DoubleType()),
    StructField("Education", DoubleType()),
    StructField("Income", DoubleType())
])


kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:29092") \
    .option("subscribe", "health_data") \
    .load()

parsed_df = kafka_df \
    .selectExpr("CAST(value AS STRING) AS jsonString") \
    .select(from_json(col("jsonString"), schema).alias("data")) \
    .select("data.*")

predictions = loaded_model.transform(parsed_df)

predictions = predictions.drop("features", "indexedFeatures", "rawPrediction", "probability")

result_json_df = predictions \
    .selectExpr("to_json(struct(*)) AS value")

stream_writer = result_json_df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:29092") \
    .option("topic", "health_data_predicted") \
    .outputMode("append") \
    .option("checkpointLocation", "checkpoints") \
    .start()

stream_writer.awaitTermination()

24/01/20 16:17:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/01/20 16:17:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/01/20 16:17:26 WARN DAGScheduler: Broadcasting large task binary with size 13.4 MiB
24/01/20 16:17:27 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/01/20 16:17:28 WARN DAGScheduler: Broadcasting large task binary with size 13.4 MiB
24/01/20 16:17:28 WARN DAGScheduler: Broadcasting large task binary with size 13.4 MiB
24/01/20 16:17:30 WARN DAGScheduler: Broadcasting large task binary with size 13.4 MiB
24/01/20 16:17:31 WARN DAGScheduler: Broadcasting large task binary with size 13.4 MiB
24/01/20 16:17:33 WARN DAGScheduler

KeyboardInterrupt: 